# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hawaiian paradise park,19.5933,-154.9731,24.46,92,100,2.57,US,1696097543
1,1,adamstown,-25.0660,-130.1015,20.53,88,100,11.86,PN,1696097543
2,2,port-aux-francais,-49.3500,70.2167,2.25,98,99,10.20,TF,1696097543
3,3,pitimbu,-7.4706,-34.8086,29.33,57,24,7.15,BR,1696097543
4,4,avarua,-21.2078,-159.7750,25.03,94,75,4.12,CK,1696097175


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot_humidity = city_data_df.hvplot. points("Lng", "Lat", 
                                                geo = True, 
                                                tiles = "OSM", 
                                                size = "Humidity",
                                                color = "City",
                                                color_key = "Set3", 
                                                line_color = "k"
                                                )

# Display the map
map_plot_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [42]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_vacation_city = city_data_df[(city_data_df["Humidity"] <= 80) & (city_data_df["Humidity"] > 20)]  
ideal_vacation_city = ideal_vacation_city[(ideal_vacation_city["Max Temp"] > 20.00) & (ideal_vacation_city["Max Temp"] <= 50.00)]
ideal_vacation_city = (ideal_vacation_city[ideal_vacation_city["Wind Speed"] <= 5]) 
ideal_vacation_city = (ideal_vacation_city[ideal_vacation_city["Cloudiness"] <= 30])
ideal_vacation_city

# Drop any rows with null values
ideal_vacation_city = ideal_vacation_city.dropna()


# Display sample data
ideal_vacation_city

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,madruga,22.9114,-81.8561,31.52,55,30,1.34,CU,1696097176
23,23,lata,40.1629,-8.3327,24.57,44,4,1.03,PT,1696097548
25,25,bovril,-31.3431,-59.4451,21.62,28,6,4.14,AR,1696097549
36,36,holualoa,19.6228,-155.9522,27.10,73,20,0.00,US,1696097551
44,44,lagoa,39.0500,-27.9833,20.16,72,4,4.78,PT,1696097553
...,...,...,...,...,...,...,...,...,...,...
520,520,sidi amrane,33.4989,6.0080,30.24,30,0,3.89,DZ,1696097670
551,551,la ligua,-32.4524,-71.2311,21.20,52,0,3.77,CL,1696097677
567,567,rahatgarh,23.7833,78.3667,23.75,58,4,1.08,IN,1696097681
570,570,ciudad acuna,29.3232,-100.9522,34.12,48,20,4.12,MX,1696097409


### Step 3: Create a new DataFrame called `hotel_df`.

In [43]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_vacation_city.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,madruga,CU,22.9114,-81.8561,55,
23,lata,PT,40.1629,-8.3327,44,
25,bovril,AR,-31.3431,-59.4451,28,
36,holualoa,US,19.6228,-155.9522,73,
44,lagoa,PT,39.0500,-27.9833,72,
...,...,...,...,...,...,...
520,sidi amrane,DZ,33.4989,6.0080,30,
551,la ligua,CL,-32.4524,-71.2311,52,
567,rahatgarh,IN,23.7833,78.3667,58,
570,ciudad acuna,MX,29.3232,-100.9522,48,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
madruga - nearest hotel: No hotel found
lata - nearest hotel: Residencial Botânico
bovril - nearest hotel: King' S
holualoa - nearest hotel: Kona Hotel
lagoa - nearest hotel: Casa da Madrinha
skeldon - nearest hotel: Classic International Hotel
arish - nearest hotel: سينا
bedarieux - nearest hotel: Hôtel de l'Orb
ad darb - nearest hotel: No hotel found
batesville - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
kambove - nearest hotel: No hotel found
kailua-kona - nearest hotel: Kona Seaside Hotel
north platte - nearest hotel: Howard Johnson Inn - North Platte
praia da vitoria - nearest hotel: Salles
remchi - nearest hotel: No hotel found
horta - nearest hotel: Azoris Faial Garden - Resort Hotel
mcalester - nearest hotel: Hiway Motor Inn & Suites
roma - nearest hotel: Wingate by Wyndham Rome
baiti - nearest hotel: No hotel found
cuajinicuilapa - nearest hotel: Hotel Costa Dorada
florence - nearest hotel: Old bridge Apartment
barrie - 

,City,Country,Lat,Lng,Humidity,Hotel Name
8,madruga,CU,22.9114,-81.8561,55,No hotel found
23,lata,PT,40.1629,-8.3327,44,Residencial Botânico
25,bovril,AR,-31.3431,-59.4451,28,King' S
36,holualoa,US,19.6228,-155.9522,73,Kona Hotel
44,lagoa,PT,39.0500,-27.9833,72,Casa da Madrinha
...,...,...,...,...,...,...
520,sidi amrane,DZ,33.4989,6.0080,30,Darna
551,la ligua,CL,-32.4524,-71.2311,52,Hotel Anchimallen
567,rahatgarh,IN,23.7833,78.3667,58,No hotel found
570,ciudad acuna,MX,29.3232,-100.9522,48,Hotel San Jorge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot. points("Lng", "Lat", 
                                                geo = True, 
                                                tiles = "OSM", 
                                                size = "Humidity",
                                                hover_cols = ["Hotel Name", "Country"],
                                                color = "City",
                                                color_key = "Set3", 
                                                line_color = "k"
                                                )

# Display the map
map_plot_hotels

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)